#Import the dataset from the Experimental Data Repository stored on a Google Drive

We must import the dataset from the Experimental Data Repository that is stored on the Google Drive. To do this we first mount the drive, and then copy the contents back into our working space. We can then cd into it and get to work.

In [1]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#Get the data From Drive
%cp -r drive/MyDrive/maggie ./

In [2]:
%cd maggie

/content/maggie


# Append all the yearly data together
Append all the yearly data together and store it all into one data frame. We use pandas to accomplish this and store the dates for later use

In [3]:
# Read the data as a csv file
df = pd.DataFrame()
for i in range(2016, 2023 + 1):
  yearframe = pd.read_csv(f'{i}.csv', header=None)
  df = pd.concat([df, yearframe], ignore_index=True)
dates = df.iloc[:, 0]
df.drop(columns=df.columns[0], axis=1,  inplace=True)
print(dates)

0          2016-01-01 00:00:00
1          2016-01-01 00:01:00
2          2016-01-01 00:02:00
3          2016-01-01 00:03:00
4          2016-01-01 00:04:00
                  ...         
3277435    2023-05-02 23:55:00
3277436    2023-05-02 23:56:00
3277437    2023-05-02 23:57:00
3277438    2023-05-02 23:58:00
3277439    2023-05-02 23:59:00
Name: 0, Length: 3277440, dtype: object


# KP Ground Truth Dataset Collction
We open a seperate dataset for the ground truth $k_p$ value. These scores will be compared with the model's output and used to make the model better. We test the valditity of these values below, testing if there are exactly $8$ for each day.

In [4]:
# Validate the JSON KP Value Dataset
import json
with open("kp_values.json") as f:
  jsonstring = json.load(f)
  kp_values = json.loads(jsonstring)

for date, value in kp_values.items():
  if(len(value) != 8):
    print("Something went wrong!")

# Data Processing
We must first preform some computations on the data before we end up using it for our machine learning algorithms. First we convert to a numpy array for easy processing, then for every column in the observed spectra, we normalize the value to ensure that every value is between $[0, 1]$. Any values that are $0$ are considered invalid data, and so we input $-1$ into these to indicate a seperation between these values and the rest of the data.

In [5]:
from sklearn.preprocessing import normalize

combined_dataset = df.to_numpy()
combined_dataset[np.isnan(combined_dataset).any(axis=1)] = np.zeros(53)
normalized_data = combined_dataset[:, :3]
for i in range(3, 53):
  normalized_data = np.append(normalized_data, np.divide(combined_dataset[:, i:i+1], np.max(combined_dataset[:, i:i+1])), axis=1)
normalized_data[normalized_data == 0] = -1


# Sort ground truth values into labels set
We sort the ground truth values and put them into a labels set to match the input data coming in

In [6]:
from datetime import datetime

labels = []
for day, values in sorted(kp_values.items(), key=lambda x: datetime.strptime(x[0], '%Y-%m-%d')):
  for value in values:
    labels.append(float(value))
labels = np.array(labels)

# Chunk together the data
We know organize the data into chunks of size $180$. The reason we do this is to ensure that the smallest unit of time we can make a prediction over is $3$ hours.
This makes sense since $k_p$ is only measured over a $3$ hour gap.

In [7]:
#  Split using numpy array functions
chunks = np.array(np.split(normalized_data, normalized_data.shape[0] / 180))

# Split the data into a training and testing set
We split the data into a training and testing set using a controlled parameter to determine the split. Note that we always use any validation/testing data to be future data because we want our model to generalize well into the future

In [8]:
num_training = int(len(chunks)*0.8)
training = chunks[:num_training]
testing = chunks[num_training:]

# Reshape the training and testing data
We reshape both datasets to match both X and Y as inputs to our model

In [32]:
training_x = np.reshape(training, (len(training), 180, 53, 1))
training_y = np.reshape(labels[:num_training], (len(training), 1))
testing_x = np.reshape(testing, (len(testing), 180, 53, 1))
testing_y = np.reshape(labels[num_training:len(chunks)], (len(testing), 1))

# CNN 2D Model
Now with our data, we create our first model, a 2D CNN Model as a layer of 2D Convolutional Layers and Max Pooling Layers. We finally flatten the result and output a singular value for our estimated $k_p$ value. We then test and evaluate our model based on the training and testing data.

In [9]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [17]:
model = models.Sequential()
model.add(layers.Conv2D(32, (10, 10), activation='relu', input_shape=(180, 53, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (6, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 171, 44, 32)       3232      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 85, 22, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 80, 17, 64)        73792     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 40, 8, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 38, 6, 64)         36928     
                                                                 
 flatten_1 (Flatten)         (None, 14592)            

In [19]:
from keras.optimizers import Adam
from keras import backend
backend.clear_session()
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

history = model.fit(training_x, training_y, epochs=3,
                    validation_data=(testing_x, testing_y))

model.save('2d_cnn.keras')

Epoch 1/3
456/456 [==============================] - 462s 1s/step - loss: 1.5024 - mean_absolute_error: 0.9666 - val_loss: 1.4745 - val_mean_absolute_error: 0.9444
Epoch 2/3
456/456 [==============================] - 406s 891ms/step - loss: 1.4419 - mean_absolute_error: 0.9512 - val_loss: 1.3937 - val_mean_absolute_error: 0.9414
Epoch 3/3
456/456 [==============================] - 427s 937ms/step - loss: 1.4323 - mean_absolute_error: 0.9494 - val_loss: 1.3966 - val_mean_absolute_error: 0.9405


# CNN 1D Model
We now create our second model, as a layer of Convolutional 1D Layers together with MaxPooling1D Layers. We finally flatten the result and output a singular $k_p$ value. We then test and evaluate our model.

In [10]:
training_x = np.reshape(training, (len(training), 180, 53))
training_y = np.reshape(labels[:num_training], (len(training)))
testing_x = np.reshape(testing, (len(testing), 180, 53))
testing_y = np.reshape(labels[num_training:len(chunks)], (len(testing)))

conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(3,),
                           activation='relu'),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=32,
                        kernel_size=(3,),
                        activation='relu'),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Dense(units=32, activation='relu', input_shape=(180, 53)),
    tf.keras.layers.Dense(units=1),
])

conv_model.compile(loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])


history = conv_model.fit(training_x, training_y, epochs=10, validation_data=(testing_x, testing_y))

conv_model.save("1d_cnn.keras")

Epoch 1/10
456/456 [==============================] - 9s 17ms/step - loss: 1.5177 - mean_absolute_error: 0.9812 - val_loss: 1.3846 - val_mean_absolute_error: 0.9367
Epoch 2/10
456/456 [==============================] - 8s 18ms/step - loss: 1.4856 - mean_absolute_error: 0.9726 - val_loss: 1.3598 - val_mean_absolute_error: 0.9359
Epoch 3/10
456/456 [==============================] - 6s 13ms/step - loss: 1.4801 - mean_absolute_error: 0.9699 - val_loss: 1.3616 - val_mean_absolute_error: 0.9387
Epoch 4/10
456/456 [==============================] - 8s 17ms/step - loss: 1.4783 - mean_absolute_error: 0.9703 - val_loss: 1.3631 - val_mean_absolute_error: 0.9369
Epoch 5/10
456/456 [==============================] - 6s 13ms/step - loss: 1.4776 - mean_absolute_error: 0.9709 - val_loss: 1.4041 - val_mean_absolute_error: 0.9352
Epoch 6/10
456/456 [==============================] - 8s 17ms/step - loss: 1.4767 - mean_absolute_error: 0.9721 - val_loss: 1.4650 - val_mean_absolute_error: 0.9661
Epoch 7/10

# LSTM Model
We present our LSTM Model. We unfortunatly did not save the specific training and testing data handling, but in this code snippet after that has been filled out this trains an LSTM model with 30 units on the training set and then evaluates it on the validation set

In [63]:
# training_x = ...
# training_y = ...
# testing_x = ...
# testing_y = ...

lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(30, return_sequences=False),
    tf.keras.layers.Dense(units=1)
])

lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

history = lstm_model.fit(training_x, training_y, epochs=1, validation_data=(testing_x, testing_y))

lstm_model.save('lstm.keras')

81936/81936 [==============================] - 1515s 18ms/step - loss: 1.2666 - mean_absolute_error: 0.8906


# Collect predicted data for all years
Load the desired model, and make a prediction for every year. Then append this data to a file to be used externally.

In [21]:
model = tf.keras.models.load_model('2d_cnn.keras')
prediction = model.predict(chunks)
f = open("kp_data.txt", "w")
i = 0
for date, _ in sorted(kp_values.items(), key=lambda x: datetime.strptime(x[0], '%Y-%m-%d')):
  f.write(f"{date},{labels[i]},{prediction[i][0]}\n")
  i += 1
f.close()

569/569 [==============================] - 127s 224ms/step
